In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
state = input()

layoff = pd.read_csv('layofffinal.csv') # reads csv

#filters out alabama birmingham for testing
state_filtered = layoff[(layoff['state'] == state)]

#prints
state_filtered

Wisconsin


FileNotFoundError: ignored

In [ ]:
# filters null
state_filtered = state_filtered.interpolate()

state_filtered = state_filtered.drop(columns = ['state', 'city', 'month', 'year'])

state_filtered

,Unnamed: 0,laid_off,date
36674,36674,60.0,2021-12-30
36675,36675,81.0,2021-11-29
36676,36676,27.0,2021-11-22
36677,36677,61.0,2021-10-04
36678,36678,82.0,2021-09-30
...,...,...,...
41189,41189,74.0,2022-10-03
41190,41190,53.0,2022-08-17
41191,41191,210.0,2022-08-30
41192,41192,25.0,2022-07-26


In [ ]:
def window_input_output(input_length: int, output_length: int, data: pd.DataFrame) -> pd.DataFrame:
    
    df = data.copy()
    
    i = 1
    while i < input_length:
        df[f'x_{i}'] = df['laid_off'].shift(-i)
        i = i + 1
        
    j = 0
    while j < output_length:
        df[f'y_{j}'] = df['laid_off'].shift(-output_length-j)
        j = j + 1
        
    df = df.dropna(axis=0)
    
    return df


seq_df = window_input_output(int(0.2*len(state_filtered)), int(0.2*len(state_filtered)), state_filtered)

seq_df


<ipython-input-77-46dc6fecfd2d>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'x_{i}'] = df['laid_off'].shift(-i)
<ipython-input-77-46dc6fecfd2d>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'y_{j}'] = df['laid_off'].shift(-output_length-j)


,Unnamed: 0,laid_off,date,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,y_122,y_123,y_124,y_125,y_126,y_127,y_128,y_129,y_130,y_131
36674,36674,60.0,2021-12-30,81.0,27.0,61.0,82.0,51.0,56.0,62.0,...,43.0,34.0,55.0,49.0,47.0,34.0,49.0,42.0,27.0,54.0
36675,36675,81.0,2021-11-29,27.0,61.0,82.0,51.0,56.0,62.0,94.0,...,34.0,55.0,49.0,47.0,34.0,49.0,42.0,27.0,54.0,45.0
36676,36676,27.0,2021-11-22,61.0,82.0,51.0,56.0,62.0,94.0,88.0,...,55.0,49.0,47.0,34.0,49.0,42.0,27.0,54.0,45.0,49.0
36677,36677,61.0,2021-10-04,82.0,51.0,56.0,62.0,94.0,88.0,61.0,...,49.0,47.0,34.0,49.0,42.0,27.0,54.0,45.0,49.0,62.0
36678,36678,82.0,2021-09-30,51.0,56.0,62.0,94.0,88.0,61.0,28.0,...,47.0,34.0,49.0,42.0,27.0,54.0,45.0,49.0,62.0,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37070,37070,63.0,2018-11-26,75.0,115.0,80.0,138.0,58.0,97.0,159.0,...,62.0,38.0,210.0,33.0,47.0,34.0,31.0,93.0,118.0,74.0
37071,37071,75.0,2018-11-13,115.0,80.0,138.0,58.0,97.0,159.0,107.0,...,38.0,210.0,33.0,47.0,34.0,31.0,93.0,118.0,74.0,53.0
37072,37072,115.0,2018-11-09,80.0,138.0,58.0,97.0,159.0,107.0,104.0,...,210.0,33.0,47.0,34.0,31.0,93.0,118.0,74.0,53.0,210.0
37073,37073,80.0,2018-11-08,138.0,58.0,97.0,159.0,107.0,104.0,65.0,...,33.0,47.0,34.0,31.0,93.0,118.0,74.0,53.0,210.0,25.0


In [ ]:
X_cols = [col for col in seq_df.columns if col.startswith('x')]

X_cols.insert(0, 'laid_off')

y_cols = [col for col in seq_df.columns if col.startswith('y')]
X_train = seq_df[X_cols][:-2].values
y_train = seq_df[y_cols][:-2].values

X_test = seq_df[X_cols][-2:].values
y_test = seq_df[y_cols][-2:].values

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_seq = DecisionTreeRegressor(random_state=42)

dt_seq.fit(X_train, y_train)

dt_seq_preds = dt_seq.predict(X_test)

dt_seq_preds

array([[1.546e+03, 1.280e+02, 1.310e+02, 3.000e+00, 4.900e+01, 1.097e+03,
        7.000e+01, 3.930e+02, 2.180e+02, 1.420e+02, 3.800e+01, 5.300e+01,
        2.900e+01, 1.113e+03, 2.150e+02, 1.000e+00, 6.100e+01, 7.100e+01,
        3.900e+01, 1.500e+01, 5.300e+01, 5.420e+02, 3.810e+02, 1.230e+02,
        4.800e+01, 1.530e+02, 2.190e+02, 5.000e+01, 5.570e+02, 1.053e+03,
        1.720e+02, 2.600e+01, 3.700e+01, 1.100e+03, 8.900e+01, 9.300e+01,
        3.000e+02, 1.170e+02, 6.390e+02, 8.290e+02, 2.500e+01, 2.130e+02,
        2.800e+01, 2.800e+01, 5.620e+02, 6.000e+01, 4.600e+01, 5.600e+01,
        3.100e+01, 6.700e+01, 3.400e+01, 1.090e+02, 5.100e+01, 3.600e+01,
        2.260e+02, 3.240e+02, 9.000e+00, 9.000e+00, 1.300e+01, 2.470e+02,
        3.000e+01, 6.300e+01, 4.200e+01, 1.400e+02, 2.800e+01, 3.600e+01,
        2.600e+02, 2.000e+02, 3.190e+02, 1.100e+01, 3.050e+02, 4.900e+01,
        2.400e+01, 6.900e+01, 7.000e+01, 6.500e+01, 7.800e+01, 6.800e+01,
        4.300e+01, 2.900e+01, 4.400e+0

In [ ]:
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import GradientBoostingRegressor

gbr_seq = GradientBoostingRegressor(random_state=42)

chained_gbr = RegressorChain(gbr_seq)

chained_gbr.fit(X_train, y_train)

gbr_seq_preds = chained_gbr.predict(X_test)

gbr_seq_preds

In [ ]:
def mape(y_true, y_pred):
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)

mape_dt_seq = mape(dt_seq_preds.reshape(1, -1), y_test.reshape(1, -1))
mape_gbr_seq = mape(gbr_seq_preds.reshape(1, -1), y_test.reshape(1, -1))
mape_baseline = mape(X_test.reshape(1, -1), y_test.reshape(1, -1))
# Generate the bar plot
fig, ax = plt.subplots()

x = ['Baseline', 'Decision Tree', 'Gradient Boosting']
y = [mape_baseline, mape_dt_seq, mape_gbr_seq]

ax.bar(x, y, width=0.4)
ax.set_xlabel('Regressor models')
ax.set_ylabel('MAPE (%)')
ax.set_ylim(0, 1)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 0.05, s=str(value), ha='center')
    
plt.tight_layout()